# Using Insitu Data with FluxEngine #

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
from mpl_toolkits.basemap import Basemap
from netCDF4 import Dataset
# Install basemap-data-hires

In [ ]:
# Load data file
region_data = pd.read_csv('CarrickRoads.tsv', sep='\t', index_col=0)
# Show small proportion of the data
region_data.head(5)

In [ ]:
# Initialise the new Dataframe column and fill with a hold value
region_data['Hours_since'] = 'hold value'

# Produce a datetime object for the first recording 
# - the zeros in the line below show it's the first row (index starts at zero)
start_date = dt.datetime(region_data.loc[0,'Year'],region_data.loc[0,'Month'],region_data.loc[0,'Day'],
                            region_data.loc[0,'Hour'],region_data.loc[0,'Minute'],region_data.loc[0,'Second'])

# Loop over all rows in the Dataframe - i.e from 0 to the length of the Dataframe
for i in range(0,len(region_data)):
    # Get the date time object for the currently indexed recording - indexed by i
    future_date = dt.datetime(region_data.loc[i,'Year'],region_data.loc[i,'Month'],region_data.loc[i,'Day'],
                              region_data.loc[i,'Hour'],region_data.loc[i,'Minute'],region_data.loc[i,'Second'])
    
    # Find difference between current datetime and inital datetime
    day_diff = future_date - start_date
    
    # Fill Dataframe column with time difference in seconds (found using .total_seconds()) 
    # divided by 86400 (proportion of days that have passed)
    region_data.loc[i,'Hours_since'] = day_diff.total_seconds()/(60*60)

In [ ]:
# Filter data to 'Datetime' and 'Days_since' columns and show first 5 rows.
region_data[['Datetime', 'Hours_since']].head(5)

In [ ]:
# Get min and max of longitude and latitude
region_data.describe().loc[['min','max'],['Lon','Lat']]

In [ ]:
def get_coords(location):
    if location == 'CarrickRoads':
        lon_min = -5.2
        lon_max = -4.9
        lat_min = 50.1
        lat_max = 50.25
        return lon_min, lon_max, lat_min, lat_max
    
    elif location == 'Agulhas':
        lon_min = 19.7
        lon_max = 20.2
        lat_min = -35.0
        lat_max = -34.7
        return lon_min, lon_max, lat_min, lat_max
    
    else:
        lon_min = input('Enter minimum longitude (most Westerly): ')
        lon_max = input('Enter minimum longitude (most Easterly): ')
        lat_min = input('Enter minimum latitude (most Southerly): ')
        lat_max = input('Enter maximum latitude (most Northerly): ')
        print('\n\n')
        if (lon_min >= lon_max) or (lat_min >= lat_max):
            print('Check if min/max were entered in the correct order (is a min greater than a max?)')
            return np.nan, np.nan, np.nan, np.nan
        
        
        return float(lon_min), float(lon_max), float(lat_min), float(lat_max)

In [ ]:
### Change string to 'CarrickRoads', 'Agulhas' or your own region name (use '_' for spaces)
region_name = 'CarrickRoads'
# Performs function
lon_min, lon_max, lat_min, lat_max = get_coords(region_name)

# Note: if you're having problems with the input fields you can uncomment the line below 
# and  just enter the values instead, but also comment out the line above to avoid confusion.

# lon_min, lon_max, lat_min, lat_max = __ , __ , __ , __

# Print out current values
print('Current values:')
print(f'Longitude -> \t min:{lon_min} \t max:{lon_max}')
print(f'Latitude -> \t min:{lat_min} \t max:{lat_max}')

In [ ]:
split_lon = round((lon_max - lon_min)/0.05) + 1
lons = np.linspace(lon_min,lon_max,split_lon)

split_lat = round((lat_max - lat_min)/0.05) + 1
lats = np.linspace(lat_min,lat_max,split_lat)

### Adding Satelite Sea Surface Sub-skin Temperature
To use FluxEngine we need the sea surface sub-skin temperature, which can be aquired in daily resolution from the ESA CCI datasets (found here: https://resources.marine.copernicus.eu/product-detail/SST_GLO_SST_L4_REP_OBSERVATIONS_010_024/INFORMATION). Under the 'Data Access' tab there are two options 'ESACCI-GLO-SST-L4-REP-OBS-SST' which covers 1981-2016, and 'C3S-GLO-SST-L4-REP-OBS-SST' for 2017 onwards. By far the easiest way to download the data is using FTP (File Transfer Protocol) which opens the file folders remotely on your PC. Some browsers (particularlly Safari on MacOS) don't allow FTP, but this can be worked around by using a different browser, such as Firefox. You should be able to connect using the 'guest' option when it prompts you, but if you can't try creating an account with the CMEMS (https://resources.marine.copernicus.eu/registration-form) and using your login at the prompt.

The resolution of the ESA CCI data is 0.05x0.05, which results in 25,920,000 data points where most are not needed. You can use the entire dataset with FluxEngine, but it takes a long time to run and also presents problems viewing the data in Panoply. Therefore the following section of code allows us to view only the data in our desired region, and then we can manually fill this in to our dataframe. 

Below we first read the netCDF SST file (you can subsitute the file string with your own downloaded ESA CCI SST data when needed)

In [ ]:
sst_ds = Dataset('20130903120000-ESACCI-L4_GHRSST-SSTdepth-OSTIA-GLOB_CDR2.0-v02.0-fv01.0.nc')

Now we want to find the SST values just within region we are interested (i.e the region plotted above). The code below takes our min/max longitudes and latitudes and finds where their array position is in the ESA CCI SST data. 

In [ ]:
# Finding SST array index for min/max longitude
lon_min_idx = round((lon_min+180)/0.05)
lon_max_idx = round((lon_max+180)/0.05)

# Finding SST array index for min/max latitude
lat_max_idx = round(-(lat_max-90)/0.05)
lat_min_idx = round(-(lat_min-90)/0.05)

# Print the results
print('Indexes:')
print(f'Rows: {lat_max_idx} -> {lat_min_idx}')
print(f'Cols: {lon_min_idx} -> {lon_max_idx}')
print(f'Data slice used: [{lat_max_idx}:{lat_min_idx},{lon_min_idx}:{lon_max_idx}]')

# Get ESA SST values by removing the mask - fill land areas with NaN values
sst_grid = np.ma.filled(sst_ds.variables['analysed_sst'][:],fill_value=np.nan)
sst_grid = np.squeeze(sst_grid) # Reduces data to 2 dimensions
sst_grid = np.flip(sst_grid, axis=0) # Flip data (ESA grid is 'upside down')
# Reduce data to desired region using the indexes found above
region_sst_grid = sst_grid[lat_max_idx:lat_min_idx,lon_min_idx:lon_max_idx]

# Plot the region SST data and annotate values (note: annotation not advised for large data sets)
plt.figure(figsize=(20,10))
sns.heatmap(region_sst_grid,annot=True,fmt=".3f",linewidths=1,linecolor='black')

In order to not calculate fluxes for the entire globe we need to add the ESA SST data into our region dataset. This can be done prgrammatically or using software like Excel. The big problem with using Excel is that when you import your data as a spreadsheet, Excel will change the format of the datetime column - but we need it as yyyy-mm-dd hh:mm:ss for use with FluxEngine - and then we have to go through the convoluted (but possible) process of changing the format back. Instead there is some code below which extracts the array position of the recorded data, matches this to the region SST grid, and fills in the values for you. This code can be complex to write initially, but will likely save time and effort in the long run. Try running the code below using the example datasets, and then test your own.

First we edit our region dataframe to include the 'ESA_SST' column and fill it with a holding value.

In [ ]:
# Add ESA_SST column and fill with 'hold value'
region_data['ESA_SST'] = 'hold value'
# Display small section of dataframe
region_data[['Datetime','ESA_SST']].head(5)

In [ ]:
# Flip the order of the latitudes (matches latitude index with SST array index)
flip_lats = np.flip(lats,axis=0)

# Loop over rows in region dataframe
for r,row in enumerate(region_data.iterrows()):
    lat = row[1]['Lat'] # Get latitude in row
    lon = row[1]['Lon'] # Get longitude in row
    
    # Loop over increasing longitude values
    # When our 'lon' is less than the longitude we are looping over we know the array index
    for lx in range(1,len(lons),1):
        if lon < lons[lx]:
            break # Breaks out of loop to keep 'lx' = array index
            
    # Loop over increasing latitude values
    # When our 'lat' is less than the latitude we are looping over we know the array index      
    for ly in range(len(flip_lats)-2,-1,-1):
        if lat < flip_lats[ly]:
            break # Breaks out of loop to keep 'ly' = array index
    
    # In the current row we changes the ESA_SST hold value to the correct value in the SST array
    region_data.loc[r,'ESA_SST'] = region_sst_grid[ly,lx-1]

In [ ]:
# Visual check of top of data (early in datetime)
region_data[['Lat','Lon','Datetime','ESA_SST']].head(5)

In [ ]:
# Visual check of bottom of data (late in datetime)
region_data[['Lat','Lon','Datetime','ESA_SST']].tail(5)

Finally we need to export our new dataframe as a file to be used in FluxEngine (change the file name to suit the dataset you are using) - be careful NOT to overwrite your orginal data just in case.

In [ ]:
# Exports data to a TSV - automatically takes name from the one chosen previously
region_data.to_csv(f'{region_name}_withESASST.tsv', sep='\t')

### Using FluxEngine
Now we have plotted our data we can use FluxEngine to export the data to netCDF files and run FluxEngine (as in tutorial 02).

Note: don't forget the -h label if you need a reminder of how the commands work

Due to how terminal commands are executed this has to be filled out by hand. The commands for the example dataset have been provided (and I suggest looking through all the included commands), but you will need to write your own (or edit the examples) for your own data.

When using the example datasets you will need to change the input file (first string), -ncOutPath, and the --limits commands. When using your own datasets you will need to check all command inputs, and change values where needed.

(Please also check your datetime format as anything except yyyy-mm-dd hh:mm:ss may cause issues - defintely double check the format if you've used Excel at any point).

In [ ]:
# Displays values needed for --limit command below
print(f'These are the values for the limit commands -> South:{lat_min}, North:{lat_max}, West:{lon_min}, East:{lon_max}')
print(f'e.g. --limits {lat_min} {lat_max} {lon_min} {lon_max}')

In [ ]:
# We primarily use FluxEngine from the command line, but here we can import it just to check the version
import fluxengine as fe
print(fe.__version__)

The following cell converts our text file to netCDF for use with FluxEngine.
If changing location to the Agulhas (or your own region) ensure to change the following:
- Text file to read (first argument)
- Coordinates (in order of South, North, West, East) given after --limits
- File name to write out netCDF to given after --ncOutPath
- If using your own data may need to change start and end times too 

In [ ]:
# Converting to netCDF
!fe_text2ncdf.py "CarrickRoads_withESASST.tsv" --limits 50.1 50.25 -5.2 -4.9 --ncOutPath "CarrickRoads.nc" --startTime "2013-09-03 00:00:00" --endTime "2013-09-03 23:59:59" --temporalResolution "30 00:00" --cols "SST_C" "windspeed" "wind_moment2" "air_pressure" "salinity" "xCO2air" "fCO2" "ESA_SST" --dateIndex 3 --latProd "Lat" --lonProd "Lon" --latResolution 0.005 --lonResolution 0.005


Here we load the configuration file and tell FluxEngine our start and end dates. Change config file if using a different region, and change dates if required.

In [ ]:
# Running FluxEngine
!fe_run.py "CarrickRoads.conf" -s "2013-09-01" -e "2013-09-30" -l


Finally we can merge our FluxEngine output into our text file. Depending on region, you may need to change the first 3 arguments:
- 1st: FluxEngine output file
- 2nd: Text file to merge to
- 3rd: New file to create with merged data

In [ ]:
# Appending FluxEngine results
!fe_append2insitu.py "fe_output/carrickroads/2013/09/carrickroads_fe.nc" "CarrickRoads.tsv" "CarrickRoads_merged.tsv" --varsToAppend "OF" "OK3" "OSFC" "OIC1" --dateIndex 3 --lonCol "Lon" --latCol "Lat"


Now we have run FluxEngine and combine our output with our in-situ data, we can go ahead and view/visualise the data.

In [ ]:
# Load merged data
merged_data = pd.read_csv('CarrickRoads_merged.tsv', sep='\t',index_col=0).reset_index(drop=True)

In [ ]:
# View top of merged data
merged_data.head()

We need to add out 'Days_since' to this new merged dataframe:

In [ ]:
# Initialise the new Dataframe column and fill with a hold value
merged_data['Hours_since'] = 'hold value'

# Produce a datetime object for the first recording 
# - the zeros in the line below show it's the first row (index starts at zero)
start_date = dt.datetime(merged_data.loc[0,'Year'],merged_data.loc[0,'Month'],merged_data.loc[0,'Day'],
                            merged_data.loc[0,'Hour'],merged_data.loc[0,'Minute'],merged_data.loc[0,'Second'])

# Loop over all rows in the Dataframe - i.e from 0 to the length of the Dataframe
for i in range(0,len(merged_data)):
    # Get the date time object for the currently indexed recording - indexed by i
    future_date = dt.datetime(merged_data.loc[i,'Year'],merged_data.loc[i,'Month'],merged_data.loc[i,'Day'],
                              merged_data.loc[i,'Hour'],merged_data.loc[i,'Minute'],merged_data.loc[i,'Second'])
    
    # Find difference between current datetime and inital datetime
    day_diff = future_date - start_date
    
    # Fill Dataframe column with time difference in seconds (found using .total_seconds()) 
    # divided by 86400 (proportion of days that have passed)
    merged_data.loc[i,'Hours_since'] = day_diff.total_seconds()/(60*60)

In [ ]:
# Show section of 'Days_since' column for visual check
merged_data[['Datetime', 'Hours_since']].head(5)

### Plot the Flux

In [ ]:
# Set up a figure with 4 axes on it. Sharex=True means all axes will share the bottom axes (can help with clarity)
fig,ax = plt.subplots(1,1, sharex=True, figsize=(20,10))
sns.lineplot(data=merged_data, x='Hours_since', y='OF [g C m-2 day-1]')

# Plot features 
plt.xlabel(f'Hours since {merged_data.loc[0,"Datetime"]}', fontdict={'size':20})
plt.ylabel('Flux [g C m-2 day-1]', fontdict={'size':20})
plt.tick_params(labelsize=15)

# Show figure!
plt.show()